## PDF文档处理

In [2]:
import PyPDF2
import re

def extract_pdf_text(pdf_path):
    """提取PDF文本内容"""
    text = ""
    with open(pdf_path, "rb") as file:
        reader = PyPDF2.PdfReader(file)
        for page in reader.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text
    return text

def clean_pdf_text(text):
    """清洗PDF文本"""
    # 移除页眉页脚
    cleaned_text = re.sub(r"第\s*\d+\s*页", "", text)
    # 压缩空白字符
    cleaned_text = re.sub(r"\s+", " ", cleaned_text).strip()
    return cleaned_text

# 使用示例
pdf_path = "./document.pdf"
raw_text = extract_pdf_text(pdf_path)
print("原始文本:"+ raw_text)
print("处理后:" + clean_pdf_text(raw_text))

原始文本:第1页  
合同编号： XYZ-2025 
甲方：ABC公司... 
 
  第2页 1.1条款... 
处理后:合同编号： XYZ-2025 甲方：ABC公司... 1.1条款...


## TXT文档处理

In [1]:
import re

def read_txt_with_encoding(txt_path):
    """多编码尝试读取TXT文件"""
    encodings = ["utf-8", "gbk", "latin-1"]
    for enc in encodings:
        try:
            with open(txt_path, "r", encoding=enc) as f:
                return f.read()
        except UnicodeDecodeError:
            continue
    raise ValueError("无法识别编码")

def clean_txt_text(text):
    """清洗文本内容"""
    # 移除空行
    lines = [line.strip() for line in text.split("\n") if line.strip()]
    cleaned_text = " ".join(lines)
    # 过滤特殊字符
    cleaned_text = re.sub(r"[^\w\s\u4e00-\u9fa5]", "", cleaned_text)
    return cleaned_text

# 使用示例
txt_path = "./data.txt"
raw_text = read_txt_with_encoding(txt_path)
print("原始文本:" + raw_text)
print("处理后:" +  clean_txt_text(raw_text))

原始文本:# 第一部分：UTF-8 编码内容（中文+英文+空格）
这是一段正常的中文文本。  
Hello, World!  
这里包含一些空行：  


（此处有多个空行）  
接下来是特殊符号测试：@#$%^&*()_+  

# 第二部分：GBK 编码内容（可能出现乱码，需用 GBK 解码）  
如果用 UTF-8 打开这段文字，可能会显示乱码：  
比如：你好，世界！  
但用 GBK 编码保存时，这些中文会正确显示。  

# 第三部分：Latin-1 编码内容（英文为主）  
Latin-1 encoding test: ñ, ü, é  
This is a line with Latin characters.
处理后: 第一部分UTF8 编码内容中文英文空格 这是一段正常的中文文本 Hello World 这里包含一些空行 此处有多个空行 接下来是特殊符号测试_  第二部分GBK 编码内容可能出现乱码需用 GBK 解码 如果用 UTF8 打开这段文字可能会显示乱码 比如你好世界 但用 GBK 编码保存时这些中文会正确显示  第三部分Latin1 编码内容英文为主 Latin1 encoding test ñ ü é This is a line with Latin characters


## Word文档处理

In [11]:
from docx import Document

def extract_word_content(word_path):
    """提取Word结构化内容"""
    doc = Document(word_path)
    content_list = []
  
    for para in doc.paragraphs:
        para_text = para.text.strip()
        if not para_text:
            continue
          
        style_name = para.style.name
        if style_name.startswith("Heading"):
            # 标题处理
            content_list.append({
                "type": "heading",
                "level": int(style_name[-1]),
                "content": para_text
            })
        else:
            # 正文处理
            content_list.append({
                "type": "text",
                "content": para_text
            })
  
    return content_list

# 使用示例
word_path = "./report.docx"
content = extract_word_content(word_path)
print(content)

[{'type': 'text', 'content': '项目总结报告'}, {'type': 'text', 'content': '本项目自2023年1月启动...'}, {'type': 'text', 'content': '成果展示'}]


## Markdown文档处理

In [15]:
import re
from bs4 import BeautifulSoup
import markdown2

def md_to_clean_text(md_content):
    """正则清洗Markdown"""
    # 清除标题标记
    text = re.sub(r'^#+\s', '', md_content, flags=re.MULTILINE)
    # 清除列表标记
    text = re.sub(r'^\s*[-*0-9.]+\s', '', text, flags=re.MULTILINE)
    # 清除链接
    text = re.sub(r'\[.*?\]\(.*?\)', '', text)
    # 清除代码块
    text = re.sub(r'```[\s\S]*?```', '', text)
    # 合并空格
    text = re.sub(r'\s+', ' ', text).strip()
    return text


def md_to_html_text(md_content):
    """通过HTML转换处理Markdown"""
    html = markdown2.markdown(md_content)
    soup = BeautifulSoup(html, 'html.parser')
    return soup.get_text(separator=' ', strip=True)

def md_keep_structure(md_content):
    """保留Markdown层级结构"""
    lines = md_content.split('\n')
    result = []
  
    for line in lines:
        line = line.strip()
        if line.startswith('#'):
            level = line.count('#')
            title = line.lstrip('#').strip()
            result.append(f"{'  '*(level-1)}{title}")
        elif line:
            result.append(line)
  
    return '\n'.join(result)

# 测试
if __name__ == "__main__":
    md_path = "./test.md"
    with open(md_path, 'r', encoding='utf-8') as f:
        md_content = f.read()
    print("正则清洗:\n" + md_to_clean_text(md_content))
    print("HTML转换:\n" + md_to_html_text(md_content))
    print("保留层级结构:\n" + md_keep_structure(md_content))



    

正则清洗:
第一章 入门 **Markdown 是轻量级标记语言**，常用于写文档。
HTML转换:
第一章 入门 Markdown 是轻量级标记语言 ，常用于写文档。 python  
print("Hello, Markdown!")
保留层级结构:
第一章 入门
**Markdown 是轻量级标记语言**，常用于写文档。
```python
print("Hello, Markdown!")
```
